In [1]:
import pandas as pd
import numpy as np 
from tqdm.notebook import tqdm_notebook

In [2]:
data = pd.read_csv('data/headlines/CLUSTERED_PART_1.csv')

In [3]:
data.shape

(692347, 7)

In [4]:
#Lets check how picky can we be (how many headlines for  each week we have so far...

In [5]:
data['date'] = pd.to_datetime(data['date'])

In [6]:
start_week  = pd.to_datetime('1986-01-06')
week_starts = [start_week]
for i in tqdm_notebook(range(0,1848)):
    start_week += pd.Timedelta(days = 7)
    week_starts.append(start_week)

  0%|          | 0/1848 [00:00<?, ?it/s]

In [7]:
counts = []
for i in tqdm_notebook(week_starts):
    start = i
    end = i + pd.Timedelta(days = 7)
    counts.append(data[(data['date'] > start) & (data['date'] < end)].shape[0])


  0%|          | 0/1849 [00:00<?, ?it/s]

In [8]:
data['words_in_headline'] = data['headline'].str.split().apply(len)

In [9]:
data = data.drop(data[data['words_in_headline'] < 3].index).reset_index(drop = True)

In [10]:
data

,Unnamed: 0,headline,date,doc_type,material_type,section,keywords,words_in_headline
0,0,INVESTORS OPEN '86 CONFIDENTLY,1986-01-02,article,News,Business Day,"['NINETEEN HUNDRED EIGHTY-FIVE', 'Stocks and B...",4
1,1,ACQUISITIONS HELPED SOME AMEX LEADERS,1986-01-02,article,News,Business Day,"['NINETEEN HUNDRED EIGHTY-FIVE', 'Stocks and B...",5
2,2,TRANSCRIPT OF REAGAN'S MESSAGE TO THE SOVIET P...,1986-01-02,article,Text,World,"['Speeches and Statements', 'United States Int...",12
3,3,A NEW ELEMENT OF VOLATILITY,1986-01-02,article,News,Business Day,"['NINETEEN HUNDRED EIGHTY-FIVE', 'FUTURES TRAD...",5
4,4,2 Arabs in Belgium Accused Of Planning an Airp...,1986-01-02,article,News,World,"['Palestinians', 'AIRPORTS', 'Arabs', 'Terrori...",10
...,...,...,...,...,...,...,...,...
676686,692342,"Russia scrambles to contain a new surge, as mo...",2021-06-12,article,News,World,"['internal-essential', 'Coronavirus (2019-nCoV...",19
676687,692343,President Biden and Jill Biden will meet with ...,2021-06-12,article,News,World,"['Royal Families', 'United States Internationa...",13
676688,692344,Yair Lapid Won’t Be Israel’s Next Leader. But ...,2021-06-12,article,News,World,"['Politics and Government', 'Legislatures and ...",14
676689,692345,Climate Protesters Rally on the Streets and at...,2021-06-12,multimedia,Video,World,"['Demonstrations, Protests and Riots', 'Global...",12


In [11]:
from nltk.corpus import wordnet as wn

In [12]:
def possible_verb(word_str):
    word = wn.synsets(word_str)
    for i in word:
        if i.pos() == 'v':
            return True
    else:
        return False

In [13]:
possible_verb('volatility')

False

In [14]:
possible_verb('codes')

True

In [15]:
possible_verb_counts = []
for headline in tqdm_notebook(data['headline']):
    possible_verbs = 0
    for word in headline.split():
        if possible_verb(word):
            possible_verbs += 1
    possible_verb_counts.append(possible_verbs)

  0%|          | 0/676691 [00:00<?, ?it/s]

In [16]:
data['possible_verbs'] = pd.Series(possible_verb_counts)

In [17]:
data = data.drop(data[data['possible_verbs'] == 0].index).reset_index(drop = True)

In [18]:
#Unify headline format

In [19]:
data['headline'] = data['headline'].str.lower()
data['headline'] = data['headline'].str.title()

In [20]:
data.sample(20)['headline']

336194          New York Gains Ground In Bid For 2012 Games
58181                            European Coin Seen Gaining
512063    Russian Court Upholds Convictions Of Imprisone...
391911    Khmer RougeS Last Bastion Calls On Spirits Fo...
160554    Kellogg May Repurchase $300 Million More Of It...
276412          Treaty'S End Would Ruffle Russia'S Politics
278254    Technology Briefing | Telecommunications: Grou...
162057        Mannesmann Acquires Imo Unit For $124 Million
158337    Katzenberg To Form Studio With Geffen And Spie...
250190          Home-Grown Drug Business Booms In Vancouver
524258    I.M.F. Trims Estimates For Global Growth On ‘S...
380218     Are Hedge Funds On Deck As Baseball Team Owners?
27640     Contra Chief Says He Gave $50,000 In Effort To...
259438                What Brings A Bear Back To The Market
230645    America Online Buys Stake In Asian Web Site Op...
506641                                  Cutting Pell Grants
228915    Blair Rallies Public Support A

In [21]:
data = data.drop(['Unnamed: 0'], axis = 1)

In [22]:
data.to_csv('data/headlines/FINAL_0_1.csv')